In [41]:
#from helpers import *
from keras.layers import *
from keras.models import Model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [10]:
print(tf.__version__)
print(tf.config.list_physical_devices())

2.7.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [11]:
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Device found : {physical_device}')

Device found : [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [12]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [13]:
tf.config.experimental.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [14]:
tf.config.experimental.get_memory_growth(physical_device[0])

In [36]:
np.random.seed(0)
tf.random.set_seed(0)

num_classes = 10
input_shape = (32, 32, 3)

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=(0,180),
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
datagen.fit(x_train)
datagen_test = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True)
datagen_test.fit(x_train)

def split_data(images, labels, train_size=0.9, shuffle=True):
    # 1. Get the total size of the dataset
    size = len(images)
    # 2. Make an indices array and shuffle it, if required
    indices = np.arange(size)
    if shuffle:
        np.random.shuffle(indices)
    # 3. Get the size of training samples
    train_samples = int(size * train_size)
    # 4. Split data into training and validation sets
    x_train, y_train = images[indices[:train_samples]], labels[indices[:train_samples]]
    x_valid, y_valid = images[indices[train_samples:]], labels[indices[train_samples:]]
    return x_train, x_valid, y_train, y_valid


# Splitting data into training and validation sets
x_train, x_valid, y_train, y_valid = split_data(x_train, y_train)

In [16]:
# ResNet building block of two layers
def building_block(X, filter_size, filters, stride=1):

    # Save the input value for shortcut
    X_shortcut = X

    # Reshape shortcut for later adding if dimensions change
    if stride > 1:

        X_shortcut = Conv2D(filters, (1, 1), strides=stride, padding='same')(X_shortcut)
        X_shortcut = BatchNormalization(axis=3)(X_shortcut)

    # First layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=stride, padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Second layer of the block
    X = Conv2D(filters, kernel_size = filter_size, strides=(1, 1), padding='same')(X)
    X = BatchNormalization(axis=3)(X)
    X = add([X, X_shortcut])  # Add shortcut value to main path
    X = Activation('relu')(X)

    return X

In [17]:
# Full model
def create_model(input_shape, classes, name):

    # Define the input
    X_input = Input(input_shape)

    # Stage 1
    X = Conv2D(filters=16, kernel_size=3, strides=(1, 1), padding='same')(X_input)
    X = BatchNormalization(axis=3)(X)
    X = Activation('relu')(X)

    # Stage 2
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)
    X = building_block(X, filter_size=3, filters=16, stride=1)

    # Stage 3
    X = building_block(X, filter_size=3, filters=32, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)
    X = building_block(X, filter_size=3, filters=32, stride=1)

    # Stage 4
    X = building_block(X, filter_size=3, filters=64, stride=2)  # dimensions change (stride=2)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)
    X = building_block(X, filter_size=3, filters=64, stride=1)

    # Average pooling and output layer
    X = GlobalAveragePooling2D()(X)
    X = Dense(classes, activation='softmax')(X)

    # Create model
    model = Model(inputs=X_input, outputs=X, name=name)

    return model

In [24]:
batch_size = 128
epochs = 60

def scheduler(epoch, lr):
    if epoch == 0:
        return lr
#     if (epoch % 10) == 0:
#         return lr * 0.32
    else:
        return lr

In [25]:
classes = y_train.shape[1]
opt = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
ResNet32 = create_model(input_shape=input_shape, classes=classes, name='ResNet32')
ResNet32.compile(optimizer=opt, loss='categorical_crossentropy', metrics = ['accuracy'])
schedule = keras.callbacks.LearningRateScheduler(scheduler)

In [26]:
history= ResNet32.fit(datagen.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=len(x_train) / batch_size, epochs=epochs, validation_data=datagen_test.flow(x_valid,y_valid,batch_size=100),validation_steps=50,callbacks=[schedule])

score = ResNet32.evaluate(datagen_test.flow(x_test, y_test,batch_size=100,shuffle=False),steps=len(x_test)/100)
print("Test accuracy:", score[1])

Epoch 1/60
351/351 [==============================] - 52s 133ms/step - loss: 1.5327 - accuracy: 0.4435 - val_loss: 2.2570 - val_accuracy: 0.3538 - lr: 0.0010
Epoch 2/60
351/351 [==============================] - 46s 130ms/step - loss: 1.0747 - accuracy: 0.6140 - val_loss: 1.5102 - val_accuracy: 0.5536 - lr: 0.0010
Epoch 3/60
351/351 [==============================] - 46s 131ms/step - loss: 0.8986 - accuracy: 0.6798 - val_loss: 1.1612 - val_accuracy: 0.6114 - lr: 0.0010
Epoch 4/60
351/351 [==============================] - 46s 132ms/step - loss: 0.7950 - accuracy: 0.7208 - val_loss: 1.3846 - val_accuracy: 0.5840 - lr: 0.0010
Epoch 5/60
351/351 [==============================] - 47s 133ms/step - loss: 0.7145 - accuracy: 0.7480 - val_loss: 1.1478 - val_accuracy: 0.6458 - lr: 0.0010
Epoch 6/60
351/351 [==============================] - 46s 129ms/step - loss: 0.6463 - accuracy: 0.7747 - val_loss: 0.9083 - val_accuracy: 0.6970 - lr: 0.0010
Epoch 7/60
351/351 [==============================] 

351/351 [==============================] - 45s 128ms/step - loss: 0.1372 - accuracy: 0.9509 - val_loss: 0.6898 - val_accuracy: 0.8234 - lr: 0.0010
Epoch 53/60
351/351 [==============================] - 46s 131ms/step - loss: 0.1427 - accuracy: 0.9487 - val_loss: 0.5110 - val_accuracy: 0.8634 - lr: 0.0010
Epoch 54/60
351/351 [==============================] - 46s 130ms/step - loss: 0.1331 - accuracy: 0.9528 - val_loss: 0.5957 - val_accuracy: 0.8456 - lr: 0.0010
Epoch 55/60
351/351 [==============================] - 46s 130ms/step - loss: 0.1317 - accuracy: 0.9530 - val_loss: 0.6302 - val_accuracy: 0.8374 - lr: 0.0010
Epoch 56/60
351/351 [==============================] - 46s 129ms/step - loss: 0.1312 - accuracy: 0.9528 - val_loss: 0.5378 - val_accuracy: 0.8604 - lr: 0.0010
Epoch 57/60
351/351 [==============================] - 45s 129ms/step - loss: 0.1249 - accuracy: 0.9550 - val_loss: 0.4426 - val_accuracy: 0.8776 - lr: 0.0010
Epoch 58/60
351/351 [==============================] - 46s

In [32]:
len(x_train)

45000

In [33]:
len(x_valid)

5000

In [40]:
tf.keras.optimizers.Adam?